# Analysis of character usage in simplified ASCII text


Find all distinct characters in simplified ASCII representation of an *Iliad*




In [ ]:
// Configure notebook
val personalRepo = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(personalRepo)


In [ ]:
import $ivy.`edu.holycross.shot.cite::xcite:4.3.0`
import $ivy.`edu.holycross.shot::ohco2:10.20.3`
import $ivy.`edu.holycross.shot::greek:5.5.1`
import $ivy.`edu.holycross.shot.mid::orthography:2.0.0`

In [ ]:
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.greek._
import edu.holycross.shot.mid.orthography._


val venetusAUrl = "https://raw.githubusercontent.com/neelsmith/summer2020nbs/master/data/vaIliad-2020i.cex"
val twins9Url = "https://raw.githubusercontent.com/neelsmith/summer2020nbs/master/data/twins9corpus.cex"
val allenUrl = "https://raw.githubusercontent.com/neelsmith/summer2020nbs/master/data/iliad-allen.cex"



In [ ]:
// create  source corpora
val twins9 = CorpusSource.fromUrl(twins9Url)
val allen = CorpusSource.fromUrl(allenUrl)
val venetusA = CorpusSource.fromUrl(venetusAUrl)

In [ ]:
/*
- tokenize, keep only lexical tokens
- make LiteraryGreekStrings from lexical tokens, drop accents and breathings
- recompose into a single stripped-down string for each line
*/
def curateNode(cn: CitableNode, siglum: String) : CitableNode = {
  if (cn.text.isEmpty){
    println("EMPTY TEXT: " + cn.urn)
    cn
  } else {

    val lexTokens = LiteraryGreekString.tokenizeNode(cn).filter(_.tokenCategory == Some(LexicalToken))
    val lgs = lexTokens.map(tkn => LiteraryGreekString(tkn.text).toLower.stripBreathingAccent.ascii)
    val simpleAscii = lgs.mkString(" ")
    CitableNode(cn.urn.addVersion(s"${siglum}_simpleascii"),simpleAscii)
  }
}


def asciiCorpus(c: Corpus, siglum: String) : Corpus = {
  Corpus(c.nodes.map(n => curateNode(n, siglum)))
}

In [ ]:
val oopsIliadNodes = twins9.nodes.filter(_.urn < CtsUrn("urn:cts:greekLit:tlg0012.tlg001.e3:9"))



In [ ]:

val oopsIliadAscii = asciiCorpus(Corpus(oopsIliadNodes), "e3")


In [ ]:
val chars = oopsIliadAscii.nodes.map(_.text).map(_.split("")).flatten.distinct
